<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml0922/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84140032/84125825 [==============================] - 2s 0us/step


In [2]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df

In [3]:
train_df = getdata("train")
test_df = getdata("test")

In [4]:
test_df

,content,sentiment
0,If there were two parts that the physically to...,1
1,"LIGHTS OF NEW YORK was the first ""all-taking"" ...",1
2,before watching this movie my thoughts were li...,1
3,As a recreational golfer with some knowledge o...,1
4,The reviews for RENDITION generally haven't be...,1
...,...,...
24995,The Cat in the Hat is just a slap in the face ...,0
24996,This old stinker makes the Flash Gordon movies...,0
24997,I rented Dark Harvest (the first one) because ...,0
24998,One look at the rating ought to tell you this ...,0


In [5]:
# Tokenize: 把每個詞彙換成一個整數
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [6]:
# tok.word_index
# tok.index_word

In [7]:
# Sequenize: 真的把文章的詞彙換成一個一個整數(過濾: 3000以外忽略)
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,10,470,5,64,123,234,10,216,12.0,2.0,68.0,8.0,9.0,85.0,33.0,66.0,253.0,8.0,46.0,4.0,58.0,511.0,99.0,123.0,51.0,10.0,293.0,10.0,13.0,2652.0,30.0,86.0,84.0,1.0,62.0,344.0,162.0,13.0,9.0,1095.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,894,17,52,49,1376,4,60,183,147.0,5.0,27.0,3.0,272.0,179.0,2.0,32.0,318.0,2.0,362.0,589.0,3.0,144.0,83.0,702.0,320.0,9.0,6.0,2.0,10.0,59.0,1037.0,16.0,1.0,749.0,926.0,12.0,11.0,17.0,6.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37,28,4,1,956,298,11,66,1.0,4.0,3.0,84.0,17.0,18.0,139.0,571.0,20.0,1.0,93.0,5.0,2665.0,138.0,3.0,434.0,85.0,236.0,13.0,2.0,1693.0,13.0,864.0,10.0,40.0,825.0,1061.0,15.0,9.0,5.0,76.0,125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,1,4,3,703,186,120,100,56.0,655.0,43.0,12.0,38.0,348.0,2962.0,44.0,314.0,38.0,46.0,535.0,268.0,5.0,1822.0,5.0,1257.0,9.0,53.0,1379.0,9.0,77.0,27.0,46.0,275.0,18.0,5.0,38.0,186.0,655.0,43.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29,8,29,3,49,19,2,125,15.0,1.0,189.0,12.0,66.0,1.0,19.0,21.0,74.0,90.0,1.0,62.0,235.0,2410.0,1594.0,5.0,1.0,124.0,3.0,49.0,289.0,14.0,1.0,2162.0,295.0,16.0,3.0,1594.0,498.0,2.0,1.0,1376.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,10,116,186,99,12,957,43,3,144.0,1163.0,4.0,733.0,37.0,132.0,1291.0,2195.0,60.0,10.0,255.0,5.0,27.0,176.0,49.0,18.0,128.0,13.0,1010.0,139.0,2.0,99.0,12.0,938.0,22.0,11.0,10.0,194.0,13.0,28.0,4.0,145.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,2887,3,752,5,28,4,1,88,105.0,4.0,29.0,55.0,11.0,30.0,219.0,44.0,1.0,5.0,560.0,43.0,1.0,687.0,1445.0,3.0,4.0,1.0,18.0,124.0,1742.0,1.0,2956.0,4.0,109.0,2.0,1964.0,11.0,55.0,2.0,38.0,366.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,10,232,37,204,40,293,3,19,3.0,1289.0,914.0,2.0,524.0,151.0,3.0,104.0,531.0,4.0,8.0,1.0,88.0,1119.0,2.0,29.0,278.0,4.0,1.0,678.0,858.0,564.0,2.0,1784.0,1518.0,409.0,2.0,2565.0,2565.0,2565.0,54.0,1687.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,10,1056,11,19,30,30,32,19,1329.0,9.0,13.0,615.0,100.0,1.0,705.0,1070.0,118.0,1.0,4.0,6.0,16.0,1.0,4.0,48.0,16.0,1.0,1028.0,140.0,8.0,11.0,152.0,435.0,2.0,3.0,2345.0,36.0,1.0,133.0,13.0,267.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Padding: 截長補短(補前截前)
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,9,149,1829,29,1,592,9,211,10,383,11,17,5,103,18,258,20,248,85,42,249,5,199,22,3,4,446,2,89,57,987,688,5,1,1467,39,672,588,89
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,52,49,1376,4,60,183,147,5,27,3,272,179,2,32,318,2,362,589,3,144,83,702,320,9,6,2,10,59,1037,16,1,749,926,12,11,17,6,3,2078,2525
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,96,2567,59,25,620,242,36,1,632,4,1,62,18,10,194,12,170,13,51,13,8,1,1547,264,15,1,427,11,17,735,5,25,3,2024,602,12,494,5,131,709
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,76,80,29,131,2471,439,40,84,1,223,17,1453,22,80,3,2186,1306,249,42,705,2,1,636,23,703,18,42,3,1737,5,103,9,10,59,199,4,1,462,706,155
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,1594,498,2,1,1376,6,1377,5,46,11,200,27,96,5,405,354,15,69,9,6,3,70,576,783,2,400,15,220,826,147,195,21,139,22,76,3,173,4,1028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,63,75,307,1927,17,446,2,28,50,151,8,3,186,17,1582,37,266,5,2337,3,550,16,3,456,2123,34,44,69,2,10,77,116,22,1436,395,20,1,2882,4,9
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16,86,2224,56,6,172,55,1,114,10,97,232,58,2691,529,177,80,2,1,17,44,61,28,903,344,69,22,376,1258,2,1,189,12,42,21,1,201,5,383,9
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2597,2,10,404,25,21,1631,3,110,18,10,241,4,543,2,143,803,5,64,58,698,1,872,548,1,93,58,203,124,9,149,25,5,27,18,124,9,25,5,8
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,64,14,3,459,260,1203,1029,1329,2967,16,1612,67,396,175,80,98,12,941,3,20,1,812,11,19,141,21,25,3,763,105,138,14,1061,1025,2,1105,726,227,50,2964


In [9]:
# !!!!!!!3000 + 0(padding)
INPUT_DIM = 3001
# you can try 256
OUTPUT_DIM = 128
INPUT_LENGTH = 512

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

layers = [
    # 3001(種) * 128(語意) = 384128
    Embedding(input_dim=INPUT_DIM, 
              output_dim=OUTPUT_DIM, 
              input_length=INPUT_LENGTH, 
              mask_zero=True),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [11]:
# y: 做one-hot encoding很耗費ram, 直接使用Sparse省RAM又方便
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])

In [12]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [ ]:
# batch_size: 看多少筆資料調整一次梯度(10-200)
# = 200: 54000(訓練資料) / 200 -> 270(次)梯度調整
# epochs: 54000筆看幾次
# = 10: 270 * 10 (次) 梯度調整
# validation_split: 0.1
# verbose: 1(進度條) 0(啥都沒) 2(不會有進度條)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
]
model.fit(x_train_pad,
          y_train,
          batch_size=200,
          epochs=100,
          validation_split=0.1,
          verbose=2,
          callbacks=callbacks)

In [14]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 2s 3ms/step - loss: 0.2933 - accuracy: 0.8801


[0.2933148145675659, 0.880079984664917]

In [16]:
from tensorflow.keras.layers import SimpleRNN

layers = [
    # 3001(種) * 128(語意) = 384128
    Embedding(input_dim=INPUT_DIM, 
              output_dim=OUTPUT_DIM, 
              input_length=INPUT_LENGTH, 
              mask_zero=True),
    # in1(128 * 64) + in2(64 * 64) + bias(64) = 12352
    SimpleRNN(64),
    Dense(2, activation="softmax")
]
model2 = Sequential(layers)
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                12352     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 396,610
Trainable params: 396,610
Non-trainable params: 0
_________________________________________________________________


In [17]:
# y: 做one-hot encoding很耗費ram, 直接使用Sparse省RAM又方便
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model2.compile(loss=SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
# batch_size: 看多少筆資料調整一次梯度(10-200)
# = 200: 54000(訓練資料) / 200 -> 270(次)梯度調整
# epochs: 54000筆看幾次
# = 10: 270 * 10 (次) 梯度調整
# validation_split: 0.1
# verbose: 1(進度條) 0(啥都沒) 2(不會有進度條)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
]
model2.fit(x_train_pad,
          y_train,
          batch_size=200,
          epochs=100,
          validation_split=0.1,
          verbose=2,
          callbacks=callbacks)

Epoch 1/100
113/113 - 132s - loss: 0.5564 - accuracy: 0.6877 - val_loss: 0.5050 - val_accuracy: 0.7872
Epoch 2/100
113/113 - 130s - loss: 0.3520 - accuracy: 0.8492 - val_loss: 0.3754 - val_accuracy: 0.8280
Epoch 3/100


In [ ]:
model2.evaluate(x_test_pad, y_test)